# PoC Mail Scraping API

In [1]:
!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [18]:
!pip install beautifulsoup4 pandas


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
from io import StringIO
from IPython.display import display
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import base64

In [ ]:
# Define the scope for reading Gmail emails
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

# Update this with your actual file name
CLIENT_SECRET_FILE = "client_secret_780078547466-momspjf4o5ge026ksnim4oj0e1r0asgg.apps.googleusercontent.com.json"
TOKEN_FILE = "token.json"

def authenticate_gmail():
    creds = None

    # Check if the token file exists (to avoid re-authentication)
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

    # If credentials are invalid or not available, do OAuth2 login
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the credentials for future use
        with open(TOKEN_FILE, "w") as token:
            token.write(creds.to_json())

    return build("gmail", "v1", credentials=creds)

# Authenticate and create the Gmail API service
gmail_service = authenticate_gmail()
print("✅ Gmail API Authentication Successful!")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=780078547466-momspjf4o5ge026ksnim4oj0e1r0asgg.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A49282%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=s6wYATQp2ePRRETL4NM7lIwLgWGsmg&access_type=offline
✅ Gmail API Authentication Successful!


In [ ]:
def search_emails(service, sender_email, max_results=10):
    """Search for emails from a specific sender."""
    query = f"from:{sender_email}"
    results = service.users().messages().list(userId="me", q=query, maxResults=max_results).execute()
    
    messages = results.get("messages", [])
    return messages

# Define the sender's email
sender_email = "aukehofman97@gmail.com"  # Change to your target sender

# Fetch emails from sender
emails = search_emails(gmail_service, sender_email)

# Print email IDs found
if emails:
    print("📩 Emails Found:", [email["id"] for email in emails])
else:
    print("❌ No emails found from this sender.")

📩 Emails Found: ['194f5fe95e74a426']


In [ ]:
def get_email_body(service, email_id):
    """Fetch and print the raw body of an email by its ID."""
    message = service.users().messages().get(userId="me", id=email_id, format="full").execute()
    
    # Extract email payload parts
    payload = message.get("payload", {})
    parts = payload.get("parts", [])

    email_body = None

    if parts:
        for part in parts:
            if part.get("mimeType") == "text/plain":  # Extract plain text emails
                email_body = base64.urlsafe_b64decode(part["body"]["data"]).decode("utf-8", errors="ignore")
                break
            elif part.get("mimeType") == "text/html":  # Extract HTML emails (if plain text is not available)
                email_body = base64.urlsafe_b64decode(part["body"]["data"]).decode("utf-8", errors="ignore")

    if email_body:
        print("📩 **Email Body (Raw Text):**\n")
        print(email_body)  # Print raw email content
        return email_body
    else:
        print("❌ No readable email body found.")

# Fetch and print the first email's body
if emails:
    email_id = emails[0]["id"]  # Get the first email's ID
    email_body = get_email_body(gmail_service, email_id)
else:
    print("❌ No emails found.")

📩 **Email Body (Raw Text):**



> Begin forwarded message:
> 
> From: Riku / Witronia <riku.ilomaki@witronia.com>
> Subject: Fwd: ***SPAM*** Fwd: Orders Frozen Urk 02-05 / Dry Fresh Kampen 03-05
> Date: 10 January 2025 at 13:13:05 CET
> To: aukehofman97@gmail.com
> 
> 
> 
>> Välitetty viesti alkaa:
>> 
>> Lähettäjä: Riku / Witronia <riku.ilomaki@witronia.com>
>> Aihe: Fwd: ***SPAM*** Fwd: Orders Frozen Urk 02-05 / Dry Fresh Kampen 03-05
>> Päivämäärä: 10. tammikuuta 2025 klo 14.10.04 UTC+2
>> Vastaanottaja: Wout Hofman <outlook_AF24767056900D56@outlook.com>
>> 
>>>> 
>>>>  
>>>> Good afternoon,
>>>>  
>>>> Attached you will find the new orders for delivery next week.
>>>> If possible, please provide feedback on the vehicle registration numbers on this email.
>>>>  
>>>> Please be aware that the loading this week is before 12:00 in kampen and not 19:00.
>>>>  
>>>> @Antti Koppala <mailto:antti@flowertrucks.fi> @Jere Ruokostenpohja <mailto:jere@flowertrucks.fi> Please let us know about t

In [ ]:
def extract_headers_from_text(email_body):
    """Extracts column headers from text, starting at 'Kenmerk' and ending at 'Blok DV'."""
    lines = email_body.split("\n")

    start_idx = next((i for i, line in enumerate(lines) if "Kenmerk" in line), None)
    end_idx = next((i for i, line in enumerate(lines) if "Blok DV" in line), None)

    if start_idx is None:
        print("❌ 'Kenmerk' not found in email body.")
        return None

    # If "Blok DV" is missing, assume headers go until the next empty line
    if end_idx is None:
        end_idx = next((i for i, line in enumerate(lines[start_idx:]) if line.strip() == ""), None)
        if end_idx is not None:
            end_idx += start_idx  # Adjust to absolute position

    # Extract headers
    if end_idx is not None:
        headers = [line.replace(">>>> ", "").strip() for line in lines[start_idx:end_idx+1]]
    else:
        headers = [line.replace(">>>> ", "").strip() for line in lines[start_idx:] if line.strip()]

    # Create a DataFrame with the headers as columns
    if headers:
        df = pd.DataFrame(columns=headers)
        return df
    else:
        print("❌ No headers found.")
        return None

# Extract headers into a DataFrame
df_headers = extract_headers_from_text(email_body=email_body)

# Display the DataFrame
if df_headers is not None:
    display(df_headers)
else:
    print("❌ No headers found.")

,Kenmerk,Actie,Klantnr,Naam,Land,Straat,Postcode,Plaats,Gewicht,Euro DV,Blok DV


In [65]:
prompt = """Instructions for the system settings are described here below.

Persona: 
You are an advanced Data Engineer with special expertise in scraping the web and emails. 
Moreover, you are able to perform data transformations from raw text to dataframes. 
The specific domain you do this for is the logistics domain. 
The specific use case now is for a Finnish organisation importing goods from Urk and Kampen.

Workflow of this application: 
I will give you as input a raw email that I have scraped. In the original email there was a table which is parsed as text, and needs reconstruction.
The task at hand: Take the raw text parsed from the emails, and try to reconstruct the table. 
I know your output is raw text, but I want your output to be passable into a dataframe.
The challenge: some values of the table are not filled, making it hard to fill in the values that fit the column headers.
I will give you hints in the additional information to reconstruct the table.
Desired output: a reconstructed raw output that allows me to create a dataframe, NO EXTRA TEXT or REASONING

Some additional information:
- The table starts with "Kenmerk". This is the first column header.
- The last column header is "Blok DV". From this point on, the rows with data follow afterwards.
- The action is always filled as "laden" or "lossen"
- The column "land" comes from "country" and is a country code with three letters, usually "FIN"ArithmeticError
- "Kenmerk" has 10 numbers (could be 1 more or less)
- "Euro DV" and "Blok DV" are often the empty values, or otherwise 1 or 2.
- The "straat" which is street could contain additions, like 5,b. Transform this as 5b to avoid confusion
- Always make sure the rows in the output be the same length, namely 11 (since there are 11 headers).
- ONLY RETURN THE DATA, NO EXTRA TEXT FOR REASONING
"""

In [ ]:
# Function to call the API of ChatGPT
# Load environment variables from .env file
import openai
import os
from dotenv import load_dotenv

load_dotenv()

def call_chatgpt_api(email_body, prompt):
    """
    Calls the OpenAI ChatGPT API with a prompt and email body
        
    Returns:
        str: The ChatGPT response.
    """
    api_key = os.getenv("OPENAI_API_KEY")  # Load API key securely from environment
    if not api_key:
        raise ValueError("Error: API key not found! Ensure it's set in the .env file.")

    client = openai.OpenAI(api_key=api_key)


    # Combine input into the final prompt
    full_prompt = f"""
    Given the following dataset:
    {email_body}
    
    
    And the prompt:
    {prompt}
    """

    # Call OpenAI API
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": full_prompt}
        ],
        max_tokens=1000
    )

    return response.choices[0].message.content  # Extract and return the response

In [66]:
chat_response = call_chatgpt_api(email_body, prompt)
print(chat_response)

Kenmerk,Actie,Klantnr,Naam,Land,Straat,Postcode,Plaats,Gewicht,Euro DV,Blok DV
0080305396, Lossen, 0000011637, Don Morris Palvelut Oy, FIN, Klaneettitie 12, 00420, HELSINKI, 1056, 2, 
0080305560, Lossen, 0000020523, Aminah OY, FIN, Luutnantintie 5, 00410, HELSINKI, 746, 1, 
0080305398, Lossen, D8815, Jaihe Oy, FIN, Putkitie 6, 00880, HELSINKI, 486, 1, 
0080305515, Lossen, 0000021030, Aurit Elintarvike, FIN, Hameentie 5b, 00530, HELSINKI, 795, 1, 
0080305537, Lossen, 0000013141, DPY Trading Oy, FIN, Komeetankatu 1, 02210, ESPOO, 760, 1, 
0080305424, Lossen, A16506, Xiao Hua Oy, FIN, Paalukylänpolku 9, 01600, VANTAA, 654, 1, 
0080305559, Lossen, B21812, Atusa Cash & Curry Oy, FIN, JOKELANKATU 5, 05800, HYVINKAA, 546, 1, 
0080305258, Lossen, A9354, Bai Wei Asian Shop, FIN, Kylmäsuonkatu 1, 33720, TAMPERE, 926, 1, 
0080305399, Lossen, A8613, HST Asian Market, FIN, Närpesvägen 9, 64200, NÄRPES, 856, 1, 
0080305583, Lossen, A4439, Synergy Scandic Oy, FIN, Östman Trading Oy Isokalliontie 3, 6

In [61]:
type(chat_response)

str

In [ ]:
def create_dataframe_from_csv(chat_response):
    """Converts a comma-separated string into a Pandas DataFrame."""
    # Use StringIO to simulate a CSV file from the string
    data_io = StringIO(chat_response)
    
    # Read the CSV data into a DataFrame
    df = pd.read_csv(data_io)
    
    # Display the DataFrame
    display(df)
    
    return df

# Example Usage
df_final = create_dataframe_from_csv(chat_response=chat_response)

,Kenmerk,Actie,Klantnr,Naam,Land,Straat,Postcode,Plaats,Gewicht,Euro DV,Blok DV
0,80305396,Lossen,0000011637,Don Morris Palvelut Oy,FIN,Klaneettitie 12,420,HELSINKI,1056,2,
1,80305560,Lossen,0000020523,Aminah OY,FIN,Luutnantintie 5,410,HELSINKI,746,1,
2,80305398,Lossen,D8815,Jaihe Oy,FIN,Putkitie 6,880,HELSINKI,486,1,
3,80305515,Lossen,0000021030,Aurit Elintarvike,FIN,Hameentie 5b,530,HELSINKI,795,1,
4,80305537,Lossen,0000013141,DPY Trading Oy,FIN,Komeetankatu 1,2210,ESPOO,760,1,
5,80305424,Lossen,A16506,Xiao Hua Oy,FIN,Paalukylänpolku 9,1600,VANTAA,654,1,
6,80305559,Lossen,B21812,Atusa Cash & Curry Oy,FIN,JOKELANKATU 5,5800,HYVINKAA,546,1,
7,80305258,Lossen,A9354,Bai Wei Asian Shop,FIN,Kylmäsuonkatu 1,33720,TAMPERE,926,1,
8,80305399,Lossen,A8613,HST Asian Market,FIN,Närpesvägen 9,64200,NÄRPES,856,1,
9,80305583,Lossen,A4439,Synergy Scandic Oy,FIN,Östman Trading Oy Isokalliontie 3,67600,KOKKOLA,623,1,
